First we need to load the sequence file

In [1]:
from datasets import load_dataset

trainFile="Data/s288c.fasta.train.txt"
testFile="Data/s288c.fasta.test.txt"
    

Then we will need to convert each sequence to codons, each codon is treated as a "word" and the sequence is the "sentence"

In [5]:
dataset=load_dataset('text', data_files=trainFile,split='train')
datasets = load_dataset("text", data_files={"train": trainFile, "validation": testFile})


Using custom data configuration default-f555aa9781812fb7
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-f555aa9781812fb7/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)
Using custom data configuration default-858e7f4f9c294c37
Reusing dataset text (/home/lu/.cache/huggingface/datasets/text/default-858e7f4f9c294c37/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/2 [00:00<?, ?it/s]

Now let's try to train a tokenlizer on our data

In [6]:
dataset

Dataset({
    features: ['text'],
    num_rows: 4990
})

In [7]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

In [8]:
batch_size = 1000
all_texts = [dataset[i : i + batch_size]["text"] for i in range(0, len(dataset), batch_size)]

def batch_iterator():
    for i in range(0, len(dataset), batch_size):
        yield dataset[i : i + batch_size]["text"]

In [9]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

In [10]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)


In [11]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [12]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()


In [13]:
tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")


[('This', (0, 4)),
 ('is', (5, 7)),
 ('an', (8, 10)),
 ('example', (11, 18)),
 ('!', (18, 19))]

In [14]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [15]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)


In [16]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [17]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

In [18]:
encoding = tokenizer.encode("CAA GTG ACC AGA ATG ATC ACC GGT GTT")


In [19]:
encoding.tokens

['[CLS]',
 'caa',
 'gtg',
 'acc',
 'aga',
 'atg',
 'atc',
 'acc',
 'ggt',
 'gtt',
 '[SEP]']

In [20]:
encoding.type_ids


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [27]:
tokenizer.decoder = decoders.WordPiece(prefix="##")
tokenizer.save("./yeast_token")

In [26]:
import pandas as pd
from datasets import Dataset

def tokenize_function(examples):
    return tokenizer(examples["text"])

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

# block_size = tokenizer.model_max_length
block_size = 128

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

lm_datasets =datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)


#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

TypeError: 'tokenizers.Tokenizer' object is not callable

In [ ]:
model_checkpoint = "gpt2"


tokenizer.decode(datasets["train"][1]["input_ids"])

from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)
from transformers import Trainer, TrainingArguments






In [ ]:
print(lm_datasets["train"])
print(lm_datasets["train"][0])


In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    f"{model_checkpoint}-CUBERT",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
trainer.train()